This is a example CNN(Convolutional Neural Network) I have been messing with in pytorch

In [1]:
import numpy
print(numpy.__version__)

2.0.1


Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random
import numpy as np

I am using the cifat10 dataset of 10 images types for computer vision

In [3]:
transform = transforms.ToTensor()
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True,transform=transform)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True,transform=transform)

In [4]:
cifar10_test

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

I can import data to numpy and graph it using matplotlib

In [20]:
def imageShower(x):
    randomImageInt = random.randint(0,9999)
    for x in range(x):
        image, label = cifar10_train[x+randomImageInt]
        imagenp = image.numpy()
        imagenp=np.transpose(imagenp , (1, 2, 0))
        plt.imshow(imagenp)
        plt.axis("off")
        plt.show()

In [21]:
###############################

# how many images shown in notebook
imagesShown=0

#################################
imageShower(imagesShown)

Load the dataset

In [6]:
trainloader = torch.utils.data.DataLoader(cifar10_train , batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar10_test, batch_size=128, shuffle=False)

And These are all the class names

In [7]:
classNames=cifar10_test.classes
classNames

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

The data has 3 color channel and the image is 32x32 pixels 

In [8]:
a,b = cifar10_train[0]
a.shape

torch.Size([3, 32, 32])

I defined a block of expanding channels for my CNN for simplicity

In [9]:
class LayerBlock(nn.Module):
    def __init__(self,inChannels,step):
        super(LayerBlock, self).__init__()
        half=int(inChannels+ step/2)
        self.conv = nn.Conv2d(inChannels, half, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.batch = nn.BatchNorm2d(half)
        self.conv2 = nn.Conv2d(half, inChannels+(step), kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()


    def forward(self, x):
        x=self.conv(x)
        x=self.relu(x)
        x=self.batch(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x

And make the network 6 blocks long

In [10]:
def channelPrinter(inn,out):
    print("in channels => ", inn, " and  out channels => ",out,"\n---------------------------------------------")

class nNetwork(nn.Module):
    def __init__(self):
        super(nNetwork, self).__init__()

        inChannels=3
        step=50


        channelPrinter(inChannels,inChannels+step)
        self.blockOne = LayerBlock(inChannels,step)
        inChannels=inChannels+step
        

        channelPrinter(inChannels,inChannels+step)
        self.blockTwo = LayerBlock(inChannels,step)
        inChannels=inChannels+step

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        channelPrinter(inChannels,inChannels+step)
        self.blockThree = LayerBlock(inChannels,step)
        inChannels=inChannels+step
      

        channelPrinter(inChannels,inChannels+step)
        self.blockFour = LayerBlock(inChannels,step)
        inChannels=inChannels+step
        
        
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        channelPrinter(inChannels,inChannels+step)
        self.blockFive = LayerBlock(inChannels,step)
        inChannels=inChannels+step

        channelPrinter(inChannels,inChannels+step)
        self.blockSix = LayerBlock(inChannels,step)
        inChannels=inChannels+step
        

        self.pool3 = nn.AdaptiveAvgPool2d((8, 8))
        self.dropout = nn.Dropout(p=0.5)
        self.linearLayer=nn.Linear(inChannels*8*8,256)
        self.relu = nn.ReLU()
        self.batch1d=nn.BatchNorm1d(256)
        self.linearLayer2=nn.Linear(256,10)
        self.batch1d2 = nn.BatchNorm1d(10)
        


    def forward(self, x):
        x=self.blockOne(x)
        x=self.blockTwo(x)
        x = self.pool(x)
        x = self.blockThree(x)
        x = self.blockFour(x)
        x = self.pool2(x)
        x=self.blockFive(x)
        x=self.blockSix(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.linearLayer(x)
        x = self.relu(x)
        x = self.batch1d(x)
        x=self.linearLayer2(x)
        x=self.batch1d2(x)
        return x

You can see how the neural network gets larger

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = nNetwork().to(device)

in channels =>  3  and  out channels =>  53 
---------------------------------------------
in channels =>  53  and  out channels =>  103 
---------------------------------------------
in channels =>  103  and  out channels =>  153 
---------------------------------------------
in channels =>  153  and  out channels =>  203 
---------------------------------------------
in channels =>  203  and  out channels =>  253 
---------------------------------------------
in channels =>  253  and  out channels =>  303 
---------------------------------------------


Define a criterion and optimizer

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=5e-4)

Train the model 

In [16]:
def modelTrainer(trainloader,criterion,model,optimizer):
    trainingSteps = len(trainloader)
    for epoch in range(5):
        print("epoch: ",epoch)
        for i, (images, labels) in enumerate(trainloader):
            images = images.to(device)
            labels = labels.to(device)
            modelOutput=model(images)
            loss = criterion(modelOutput, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


And test the model

In [17]:
def modelTester(testloader,model):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            modelOutput=model(images)
            _, predicted = torch.max(modelOutput.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


I have trained this model only on 5 Epochs, with raising the epochs increases both the accuracy of the model and time it takes to train the model. At higher epochs, like over 200, my model can get up to 90% accuracy. This still won't beat a pre-trained model that is fine tuned; however this project was for me to learn more about the development of machine learning models and I can take this experience with me for later projects.

In [18]:
modelTrainer(trainloader,criterion,model,optimizer)
modelTester(testloader,model)

epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
Accuracy of the network on the 10000 test images: 81.9 %
